In [ ]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set() # for plot styling
import numpy as np
import io
import folium
from sklearn.cluster import KMeans
from pygeocoder import Geocoder
import geocoder
from math import radians, sin, cos, acos, sqrt, atan2, radians
import csv

In [ ]:
def ems(df3,nc):
latit = df3['lat']
longi = df3['lng']
kmeans = KMeans(n_clusters=nc)
kmeans.fit(df3)
y_kmeans = kmeans.predict(df3)
centers = kmeans.cluster_centers_
centers = kmeans.cluster_centers_
cenlat=(centers[:,0].tolist())
cenlng=(centers[:,1].tolist())
cenlat1 = pd.Series(cenlat)
cenlng1 = pd.Series(cenlng)
cenlat2=cenlat1.rename('lat')
cenlng2=cenlng1.rename('lng')
#to set midpoint of map
a=0
b=0
mdlat=0
mdlng=0
for i in range(len(latit)):
mdlat = latit[i]+mdlat
mdlng = abs(longi[i])+mdlng

30

mdlat = mdlat/len(latit)
mdlng = mdlng/len(latit)-1

#map1
m = folium.Map(location=[40.297876, -75.581294],zoom_start=10)
for i in range(len(latit)):
folium.Marker(
location=[latit[i], longi[i]],popup="accident zone",

icon=folium.Icon(color='gray', icon='fa-plus-
square',prefix='fa')).add_to(m)

m
for i in range(len(cenlat1)):
folium.Marker(
location=[cenlat1[i], cenlng1[i]],popup="ambulance",
icon=folium.Icon(color='red', icon='fa-ambulance',prefix='fa')).add_to(m)
m

#map2
m1= folium.Map(location=[40.297876, -75.581294],zoom_start=10)
for i in range(len(latit)):
folium.Circle(
radius=50,
location=[latit[i], longi[i]],popup="accident zone",
color='crimson',
fill=True).add_to(m1)
m1
for i in range(len(cenlat1)):
folium.Circle(radius=50,location=[cenlat1[i],cenlng1[i]],popup="ambulance",
color='blue',
fill=True).add_to(m1)
m1

#map3
m2= folium.Map(location=[40.297876, -75.581294],zoom_start=10)
for i in range(len(latit)):
folium.Circle(
radius=50,
location=[latit[i], longi[i]],popup="accident zone",
color="crimson",
fill=True).add_to(m2)
for i in range(len(cenlat1)):
folium.Circle(

31

radius=100,
location=[cenlat1[i],cenlng1[i]],popup="ambulance",
color='blue',
fill=True).add_to(m2)

#to create a dual pair of centers and its points
#ccpair=[[] for x in range(len(latit))]
ccpair=[]
for i in range(len(latit)):
ccpair.append([])
min=1000
k=0
for j in range(len(cenlat)):
l=((latit[i]-cenlat[j])**2)+((longi[i]-cenlng[j])**2)
if(l<min):
min=l
k=j
ccpair[i].append(latit[i])
ccpair[i].append(longi[i])
ccpair[i].append(cenlat[k])
ccpair[i].append(cenlng[k])
pt = [tuple(x) for x in df3.to_numpy()]
cent = [tuple(x) for x in centers]
di=[]
for i in range(len(latit)):
min=1000
max=0
k=0
for j in range(len(cenlat)):
l=((latit[i]-cenlat[j])**2)+((longi[i]-cenlng[j])**2)
if(l<min):
min=l
k=j
#print(pt[i],cent[k])
points=[pt[i],cent[k]]
folium.PolyLine(points, color="blue", weight=1.5,
opacity=0.7).add_to(m2)

#largest distant point from a center
maxi=[[] for x in range(len(cenlat))]
c=0
for j in range(len(cenlat)):
max=0

32

d=0
k=0
for i in range(len(latit)):
if((cenlat[j] == ccpair[i][2] )and(cenlng[j] == ccpair[i][3])):
#print(cenlat[j],ccpair[i][0],cenlng[j],ccpair[i][1])
d=((latit[i]-cenlat[j])**2)+((longi[i]-cenlng[j])**2)
#print(d,c)
if(max<=d):
max=d
k=i
c=c+1
#print(c)
maxi[j].append(latit[k])
maxi[j].append(longi[k])
maxi[j].append(cenlat[j])
maxi[j].append(cenlng[j])

g = geocoder.osm([45,-75], method='reverse')
#address to place the ambulances
address=[]
for i in range(len(cenlat1)):
address.append(geocoder.osm([cenlat[i],cenlng[i]],method="reverse").address)
print(geocoder.osm([cenlat[i],cenlng[i]],method="reverse").address)

#plotting elbow graph
#elbow method to find the optimal no of clusters
distortions = []
K = range(1,int(nc-30))
for k in K:
kmeanModel = KMeans(n_clusters=k)
kmeanModel.fit(df3)
distortions.append(kmeanModel.inertia_)
plt.figure(figsize=(16,8))
plt.plot(K, distortions, '--bo')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

# approximate radius of earth in km
R = 6373.0

33

maxd=0
distance=0
for ad in range(len(maxi)):
lat1 = radians(maxi[ad][0])
lon1 = radians(maxi[ad][1])
lat2 = radians(maxi[ad][2])
lon2 = radians(maxi[ad][3])
dlon = lon2 - lon1
dlat = lat2 - lat1
a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
c = 2 * atan2(sqrt(a), sqrt(1 - a))
distance = R * c
if(maxd < distance):
maxd = distance
#print("\n",distance,maxd)
#print("\nThe distance is :",distance)
return m2,maxd,maxi,ccpair,latit,longi,cenlat,cenlng

In [ ]:
#maincode
#training phase
df3=pd.read_csv('Book1.csv',usecols=['lat','lng'])

ct=2
while(True):
m3,dist,maxii,ccpair,latit1,longi1,cenlat1,cenlng1=ems(df3,ct)
print("max distance when c=%d is %f" %(ct,dist))
print("\n")
if(dist < 5.333 ):
break
ct=ct+5
ct=ct-1
while(True):
m3,dist,maxii,ccpair,latit1,longi1,cenlat1,cenlng1=ems(df3,ct)
print("max distance when c=%d is %f" %(ct,dist))
if(dist> 5.33):
break
ct=ct-1

34

print("The optimal number of centers:",ct+1)

#testing of project
df4=pd.read_csv("mlproject12.csv")
latit1=df4['lat']
longi1=df4['lng']
R = 6373.0
maxd=0
c=0
distance1=0
for i in range(len(latit1)):
min=1000
max=0
k=0
for j in range(len(cenlat1)):
l=((latit1[i]-cenlat1[j])**2)+((longi1[i]-cenlng1[j])**2)
if(l<min):
min=l
k=j
lat1 = radians(latit1[i])
lon1 = radians(longi1[i])
lat2 = radians(cenlat1[k])
lon2 = radians(cenlng1[k])
dlon = lon2 - lon1
dlat = lat2 - lat1
a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
c = 2 * atan2(sqrt(a), sqrt(1 - a))
distance1 = R * c
print(distance1)
with open("Book1.csv",'a+',newline='') as
csvfile:
csvwriter = csv.writer(csvfile)
row=[]
row.append([])
row[0].append(latit1[i])
row[0].append(longi1[i])
csvwriter.writerows(row)
if(distance1 < 5.33):
points=[(latit1[i],longi1[i]),(cenlat1[k],cenlng1[k])]
folium.PolyLine(points, color="gray", weight=1.5,
opacity=0.7).add_to(m3)
else:
ct=2
while(True):
m3,dist,maxii,ccpair,latit1,longi1,cenlat1,cenlng1=ems(df3,ct)

35

#print("max distance when c=%d is %f" %(ct,dist))
#print("\n")
if(dist < 5.33 ):
break
ct=ct+5
ct=ct-1
while(True):
m3,dist,maxii,ccpair,latit1,longi1,cenlat1,cenlng1=ems(df3,ct)
#print("max distance when c=%d is %f" %(ct,dist))
if(dist> 5.33):
break
ct=ct-1
print("The optimal number of centers:",ct+1)